In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from pathlib import Path

In [18]:
entries_df = pd.read_csv(Path("../data/entries2.csv"))
entries_df["date"] = pd.to_datetime(entries_df["date"])
entries_df.loc[(entries_df['date'] > '2014-07-31') & (entries_df['date'] < '2017-06-01'), "city_name"] = "Champaign"
entries_df.loc[(entries_df['date'] < '2014-08-01'), "city_name"] = "Chicago"
entries_df.loc[(entries_df['date'] >= '2017-06-01') & (entries_df['date'] < '2020-01-01') & entries_df['city_name'].isna(), "city_name"] = "San Francisco"

weather_df = pd.read_csv(Path("../data/weather.csv"))
weather_df["date"] = pd.to_datetime(weather_df["date"])

In [19]:
entries_df.head()

,id,_hash,uuid,date,emotion,context,sentiment,sentiment_score,city_name
0,20120804053731,7e94b8882e5c9c731b703efd17fd6cae,2031417A85A7497591746749EA445757,2012-08-04 05:37:31,fear,angry|devastated|excited|proud,NEGATIVE,0.949958,Chicago
1,20120805033926,26de79ceb5312d7766a29f9d71e2850e,0C46A121B5A746D7981F412858FE3F4E,2012-08-05 03:39:26,fear,angry|anticipating|trusting|anxious|joyful,NEGATIVE,0.933307,Chicago
2,20120805180010,da5dbe3810084cdc93451bc4d8d39de1,DA284A7F3ABE46DA9BBA7D1BE3E8147E,2012-08-05 18:00:10,joy,angry|disgusted,POSITIVE,0.969270,Chicago
3,20120810180001,e5c57f8675256bed9cef0ff800a30ed7,1F9040107E1B42339FD0E7928DE98C39,2012-08-10 18:00:01,neutral,angry|devastated,POSITIVE,0.973275,Chicago
4,20120812211044,37bb3374881a12fb7d568f98a455bd96,3E541DD7F6204876BA964DDA3411C613,2012-08-12 21:10:44,neutral,angry|apprehensive|anticipating|excited|terrif...,NEGATIVE,0.983425,Chicago


In [20]:
weather_df.head()

,Unnamed: 0,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date
0,0,1293840000,2011-01-01 00:00:00 +0000 UTC,-21600,Chicago,41.878114,-87.629798,13.07,10000.0,11.97,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2011-01-01 00:00:00
1,1,1293843600,2011-01-01 01:00:00 +0000 UTC,-21600,Chicago,41.878114,-87.629798,9.54,10000.0,8.47,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2011-01-01 01:00:00
2,2,1293847200,2011-01-01 02:00:00 +0000 UTC,-21600,Chicago,41.878114,-87.629798,9.50,10000.0,8.43,...,0.20,NaN,NaN,NaN,90,500,Rain,light rain,10n,2011-01-01 02:00:00
3,3,1293850800,2011-01-01 03:00:00 +0000 UTC,-21600,Chicago,41.878114,-87.629798,8.80,10000.0,7.09,...,0.12,NaN,NaN,NaN,90,500,Rain,light rain,10n,2011-01-01 03:00:00
4,4,1293854400,2011-01-01 04:00:00 +0000 UTC,-21600,Chicago,41.878114,-87.629798,8.94,10000.0,6.73,...,NaN,NaN,NaN,NaN,90,500,Rain,light rain,10n,2011-01-01 04:00:00


In [22]:
def find_weather(date, city_name, key):
    return weather_df.loc[(weather_df["date"] >= date) & (weather_df["date"] < date + pd.DateOffset(hours=1)) & (weather_df["city_name"] == city_name), key][:1]


def get_weather(date, city_name, key):
    result = find_weather(date, city_name, key)
    if result.empty:
        return
    return result.iloc[0]

In [23]:
entries_df["clouds_all"] = entries_df.apply(lambda e: get_weather(e.date, e.city_name, "clouds_all"), axis=1)
entries_df.head()

,id,_hash,uuid,date,emotion,context,sentiment,sentiment_score,city_name,clouds_all
0,20120804053731,7e94b8882e5c9c731b703efd17fd6cae,2031417A85A7497591746749EA445757,2012-08-04 05:37:31,fear,angry|devastated|excited|proud,NEGATIVE,0.949958,Chicago,1.0
1,20120805033926,26de79ceb5312d7766a29f9d71e2850e,0C46A121B5A746D7981F412858FE3F4E,2012-08-05 03:39:26,fear,angry|anticipating|trusting|anxious|joyful,NEGATIVE,0.933307,Chicago,40.0
2,20120805180010,da5dbe3810084cdc93451bc4d8d39de1,DA284A7F3ABE46DA9BBA7D1BE3E8147E,2012-08-05 18:00:10,joy,angry|disgusted,POSITIVE,0.969270,Chicago,40.0
3,20120810180001,e5c57f8675256bed9cef0ff800a30ed7,1F9040107E1B42339FD0E7928DE98C39,2012-08-10 18:00:01,neutral,angry|devastated,POSITIVE,0.973275,Chicago,75.0
4,20120812211044,37bb3374881a12fb7d568f98a455bd96,3E541DD7F6204876BA964DDA3411C613,2012-08-12 21:10:44,neutral,angry|apprehensive|anticipating|excited|terrif...,NEGATIVE,0.983425,Chicago,75.0


In [24]:
entries_df["weather_main"] = entries_df.apply(lambda e: get_weather(e.date, e.city_name, "weather_main"), axis=1)

In [25]:
entries_df.head()

,id,_hash,uuid,date,emotion,context,sentiment,sentiment_score,city_name,clouds_all,weather_main
0,20120804053731,7e94b8882e5c9c731b703efd17fd6cae,2031417A85A7497591746749EA445757,2012-08-04 05:37:31,fear,angry|devastated|excited|proud,NEGATIVE,0.949958,Chicago,1.0,Clear
1,20120805033926,26de79ceb5312d7766a29f9d71e2850e,0C46A121B5A746D7981F412858FE3F4E,2012-08-05 03:39:26,fear,angry|anticipating|trusting|anxious|joyful,NEGATIVE,0.933307,Chicago,40.0,Clouds
2,20120805180010,da5dbe3810084cdc93451bc4d8d39de1,DA284A7F3ABE46DA9BBA7D1BE3E8147E,2012-08-05 18:00:10,joy,angry|disgusted,POSITIVE,0.969270,Chicago,40.0,Clouds
3,20120810180001,e5c57f8675256bed9cef0ff800a30ed7,1F9040107E1B42339FD0E7928DE98C39,2012-08-10 18:00:01,neutral,angry|devastated,POSITIVE,0.973275,Chicago,75.0,Clouds
4,20120812211044,37bb3374881a12fb7d568f98a455bd96,3E541DD7F6204876BA964DDA3411C613,2012-08-12 21:10:44,neutral,angry|apprehensive|anticipating|excited|terrif...,NEGATIVE,0.983425,Chicago,75.0,Clouds


In [28]:
entries_df.loc[entries_df["city_name"] == "San Francisco"]

,id,_hash,uuid,date,emotion,context,sentiment,sentiment_score,city_name,clouds_all,weather_main
189,20180625053428,3880c117a74dbb50b502d031a421edf8,7AECFE5C41DE471CBF6F1C9717B4120B,2018-06-25 05:34:28,neutral,proud,POSITIVE,0.996326,San Francisco,91.0,Clouds
190,20180625053533,b50c251df9b7c0a275c2b4a1f30ac397,AE8DC51CB1A6402CB9E774E9D66FD2F2,2018-06-25 05:35:33,disgust,anxious|sentimental|joyful|terrified|hopeful,NEGATIVE,0.978027,San Francisco,91.0,Clouds
191,20180626200649,e703c75fd71064a5c2b5526440792125,169B379A850F416FA6ABBD6EE9B4F8A7,2018-06-26 20:06:49,neutral,guilty,NEGATIVE,0.999589,San Francisco,2.0,Clear
192,20180629153609,664301a2b98f90df3c6854cbc2060e61,E4B6F3BE590F4DCDB0618F845CBEDCDC,2018-06-29 15:36:09,surprise,trusting,POSITIVE,0.992618,San Francisco,5.0,Clear
193,20180630074210,a6f1c9b38d49f04d8af30c658dc84629,4696CD3DF2694313BD10B649013B67A5,2018-06-30 07:42:10,sadness,angry|excited|anxious|trusting,NEGATIVE,0.985234,San Francisco,0.0,Clear
194,20180709031329,e5cc94ccad206677a8ea4c8093d986e5,8F31CF0AC65B4F619BCCB784A765D5E2,2018-07-09 03:13:29,sadness,angry|disgusted|anxious|trusting,NEGATIVE,0.985182,San Francisco,1.0,Clear
195,20180710154751,a63d7ee271a70aec9b98a0acc3a8ce05,39D2109F966D458B87D94F2BC850B82D,2018-07-10 15:47:51,neutral,sentimental|joyful|angry|annoyed|excited|trusting,NEGATIVE,0.990710,San Francisco,1.0,Clear
197,20180726204959,df199a16ecbd08ee7ac266eeb1d93955,2A709313630E4BB8A5FD1ABEFF4E57B1,2018-07-26 20:49:59,neutral,angry|terrified|anticipating,NEGATIVE,0.958168,San Francisco,42.0,Clouds
198,20180727003323,635bb2fb06e1384f8277a71181aaaa8b,387B7F2681914745AA59FB0F87D61097,2018-07-27 00:33:23,sadness,anxious|apprehensive|devastated|hopeful|angry|...,NEGATIVE,0.988764,San Francisco,34.0,Clouds
199,20180729170630,0f2254cf9ae63edc0f770c0af16671e8,2C1AD2EB90274C82A857F91C93BA4158,2018-07-29 17:06:30,neutral,angry|content|terrified|trusting|disgusted,NEGATIVE,0.986294,San Francisco,68.0,Clouds


In [30]:
weather_df.to_csv("/home/jovyan/work/research/data/weather.csv")
entries_df.to_csv("/home/jovyan/work/research/data/entries_with_weather.csv")